<a href="https://colab.research.google.com/github/preetamjumech/LLM/blob/main/TokenizersFromScratch_BPE_02_11_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import huggingface_hub

list(huggingface_hub.list_datasets())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[DatasetInfo(id='fka/awesome-chatgpt-prompts', author='fka', sha='459a66186f8f83020117b8acc5ff5af69fc95b45', created_at=datetime.datetime(2022, 12, 13, 23, 47, 45, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 9, 3, 21, 28, 41, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=7989, downloads_all_time=None, likes=6088, paperswithcode_id=None, tags=['task_categories:question-answering', 'license:cc0-1.0', 'size_categories:n<1K', 'format:csv', 'modality:text', 'library:datasets', 'library:pandas', 'library:mlcroissant', 'library:polars', 'region:us', 'ChatGPT'], card_data=None, siblings=None),
 DatasetInfo(id='neuralwork/arxiver', author='neuralwork', sha='698a6662e77fd5dd45dbbec988abc8123e5fa086', created_at=datetime.datetime(2024, 10, 14, 12, 21, 13, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 11, 1, 21, 18, 4, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=3207, dow

In [3]:
len(list(huggingface_hub.list_datasets()))

237315

In [4]:
from datasets import load_dataset

# Load AG News dataset
ag_news_dataset = load_dataset("ag_news", split = "train[:150]")

ag_news_dataset

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 150
})

In [5]:
ag_news_dataset['text'][1]

'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.'

In [6]:
text_samples = [example['text'] for example in ag_news_dataset]

text_samples[:5]

["Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.',
 "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.",
 'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.',
 'Oil prices soar to all-time record, posing new menace to US economy (A

In [7]:
len(text_samples)

150

In [8]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.BPE())

In [9]:
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space = False)

tokenizer.pre_tokenizer.pre_tokenize_str("We're checking pre-tokenization step.")

[('We', (0, 2)),
 ("'re", (2, 5)),
 ('Ġchecking', (5, 14)),
 ('Ġpre', (14, 18)),
 ('-', (18, 19)),
 ('tokenization', (19, 31)),
 ('Ġstep', (31, 36)),
 ('.', (36, 37))]

In [10]:
tokenizer.pre_tokenizer.pre_tokenize_str("The pièce de résistance was the chef's Special Dessert.")

[('The', (0, 3)),
 ('ĠpiÃ¨ce', (3, 9)),
 ('Ġde', (9, 12)),
 ('ĠrÃ©sistance', (12, 23)),
 ('Ġwas', (23, 27)),
 ('Ġthe', (27, 31)),
 ('Ġchef', (31, 36)),
 ("'s", (36, 38)),
 ('ĠSpecial', (38, 46)),
 ('ĠDessert', (46, 54)),
 ('.', (54, 55))]

In [11]:
trainer = trainers.BpeTrainer(vocab_size = 20000, special_tokens = ["<|endoftext|>"])

tokenizer.train_from_iterator(text_samples, trainer = trainer)

In [12]:
encoding = tokenizer.encode("We're checking pre-tokenization step.")

encoding

Encoding(num_tokens=16, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [13]:
print("Encoding_ids :", encoding.ids)

print("Encoding_tokens :", encoding.tokens)

print("Encoding_offsets :", encoding.offsets)

Encoding_ids : [2475, 2325, 95, 82, 223, 94, 972, 9, 71, 319, 92, 1330, 163, 56, 67, 10]
Encoding_tokens : ['We', "'re", 'Ġc', 'he', 'ck', 'ing', 'Ġpre', '-', 't', 'ok', 'en', 'ization', 'Ġst', 'e', 'p', '.']
Encoding_offsets : [(0, 2), (2, 5), (5, 7), (7, 9), (9, 11), (11, 14), (14, 18), (18, 19), (19, 20), (20, 22), (22, 24), (24, 31), (31, 34), (34, 35), (35, 36), (36, 37)]


In [21]:
tokenizer.post_processor = processors.ByteLevel()

# tokenizer.post_processor = processors.ByteLevel(trim_offsets = False)

In [22]:
sentence = "We're checking pre-tokenization step."

encoding = tokenizer.encode(sentence)

print("Encoding_offsets :", encoding.offsets)

Encoding_offsets : [(0, 2), (2, 5), (6, 7), (7, 9), (9, 11), (11, 14), (15, 18), (18, 19), (19, 20), (20, 22), (22, 24), (24, 31), (32, 34), (34, 35), (35, 36), (36, 37)]


In [23]:
start, end = encoding.offsets[2]

sentence[start:end]

'c'

In [24]:
start, end = encoding.offsets[9]

sentence[start:end]

'ok'

In [25]:
tokenizer.post_processor = processors.ByteLevel(trim_offsets = False)

In [26]:
sentence = "We're checking pre-tokenization step."

encoding = tokenizer.encode(sentence)

print("Encoding_offsets :", encoding.offsets)

Encoding_offsets : [(0, 2), (2, 5), (5, 7), (7, 9), (9, 11), (11, 14), (14, 18), (18, 19), (19, 20), (20, 22), (22, 24), (24, 31), (31, 34), (34, 35), (35, 36), (36, 37)]


In [27]:
start, end = encoding.offsets[2]

sentence[start:end]

' c'

In [28]:
start, end = encoding.offsets[9]

sentence[start:end]

'ok'

In [29]:
tokenizer.decoder = decoders.ByteLevel()

tokenizer.decode(encoding.ids)

"We're checking pre-tokenization step."

In [30]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object = tokenizer,
    bos_token = "<|endoftext|>",
    eos_token = "<|endoftext|>",
)

wrapped_tokenizer.tokenize("We're checking pre-tokenization step.")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


['We',
 "'re",
 'Ġc',
 'he',
 'ck',
 'ing',
 'Ġpre',
 '-',
 't',
 'ok',
 'en',
 'ization',
 'Ġst',
 'e',
 'p',
 '.']

In [31]:
from transformers import GPT2TokenizerFast

wrapped_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

wrapped_tokenizer.tokenize("We're checking pre-tokenization step.")

['We',
 "'re",
 'Ġc',
 'he',
 'ck',
 'ing',
 'Ġpre',
 '-',
 't',
 'ok',
 'en',
 'ization',
 'Ġst',
 'e',
 'p',
 '.']